In [7]:
import pandas as pd
import numpy as np
import geopandas as gp
import pickle
import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RandomizedLasso
from sklearn.linear_model import (Ridge, Lasso)
from sklearn.feature_selection import RFECV, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from minepy import MINE
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/daraperl/anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# Attempting feature selection from zip code data
df = pd.read_csv('outputs/pluto_fdny_dob_census_to_zipcode.csv')
df.fillna(0, inplace=True)

# unit area of 11359 has +inf value. removing row from data
df = df[~df['ZipCode'].isin([ 11359, 0])]

# removing columns not needed for model building
for i in ['geometry', 'AREA', 'total_gas_incidents', 'ZipCode']:
    del df[i]


# standardizing all columns to be between 0 and 1
for i in ['age', 'unit_area', 'value_per_ft', 'total_population', 'total_households', 'zip_household_mean_income', 'total_units']:
    df[i] = df[i]/max(df[i])

X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values
names = df.columns.tolist()[:-1]

In [3]:
#Run a Correlation Matrix

#df.corr()

Randomized Lasso Calculation

In [63]:
random.seed(123)
rlasso = RandomizedLasso()
rlasso.fit(X, Y)
 
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 10000), rlasso.scores_), names), reverse=True)
#All scores equal to zero. Try another model

/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call

Features sorted by their score:
[(0.34, 'DOB_dispos_G8'), (0.285, 'DOB_permit_EQ'), (0.07, 'ECB_violation_Construction'), (0.025, 'BLACK_AFRICAN_AMERICAN_ratio'), (0.02, 'DOB_dispos_A8'), (0.01, 'bldg_class_Q2'), (0.01, 'ECB_infraction_B6A'), (0.01, 'DOB_complaint_58'), (0.005, 'DOB_violation_E-ELEVATOR'), (0.0, 'zip_household_mean_income'), (0.0, 'value_per_ft'), (0.0, 'unit_area'), (0.0, 'total_units'), (0.0, 'total_population'), (0.0, 'total_households'), (0.0, 'retail_ratio'), (0.0, 'res_unit_ratio'), (0.0, 'res_ratio'), (0.0, 'office_ratio'), (0.0, 'landuse_11'), (0.0, 'landuse_10'), (0.0, 'landuse_09'), (0.0, 'landuse_08'), (0.0, 'landuse_07'), (0.0, 'landuse_06'), (0.0, 'landuse_05'), (0.0, 'landuse_04'), (0.0, 'landuse_03'), (0.0, 'landuse_02'), (0.0, 'landuse_01'), (0.0, 'com_ratio'), (0.0, 'bldg_class_Z9'), (0.0, 'bldg_class_Z8'), (0.0, 'bldg_class_Z7'), (0.0, 'bldg_class_Z5'), (0.0, 'bldg_class_Z4'), (0.0, 'bldg_class_Z3'), (0.0, 'bldg_class_Z2'), (0.0, 'bldg_class_Z1'), (0.

/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):


RFE - Recursive Feature Elimination

In [68]:
#use linear regression as the model
lr = LinearRegression()
#rank all features, i.e continue the elimination until the last ten
selector = RFECV(lr, step = 1)
selector.fit(X,Y)
 
print "Features sorted by their rank:"
print sorted(zip(map(lambda x: round(x, 4), selector.ranking_), names))

Features sorted by their rank:
[(1.0, 'bldg_class_P2'), (2.0, 'ECB_infraction_B1R'), (3.0, 'DOB_dispos_D6'), (4.0, 'bldg_class_RD'), (5.0, 'bldg_class_Q2'), (6.0, 'ECB_infraction_239'), (7.0, 'DOB_dispos_EA'), (8.0, 'bldg_class_Z1'), (9.0, 'bldg_class_W5'), (10.0, 'DOB_dispos_A6'), (11.0, 'ECB_infraction_B4C'), (12.0, 'DOB_complaint_62'), (13.0, 'ECB_infraction_B8H'), (14.0, 'ECB_infraction_274'), (15.0, 'DOB_dispos_C8'), (16.0, 'DOB_dispos_C7'), (17.0, 'ECB_infraction_B13'), (18.0, 'ECB_infraction_BB0'), (19.0, 'ECB_infraction_118'), (20.0, 'ECB_infraction_B57'), (21.0, 'ECB_infraction_127'), (22.0, 'ECB_infraction_266'), (23.0, 'ECB_infraction_B70'), (24.0, 'bldg_class_RX'), (25.0, 'bldg_class_A8'), (26.0, 'bldg_class_Y4'), (27.0, 'bldg_class_Y7'), (28.0, 'DOB_violation_LL1080-LOCAL LAW 10/80 - FACADE'), (29.0, 'bldg_class_O8'), (30.0, 'ECB_infraction_BQ3'), (31.0, 'ECB_infraction_BQ2'), (32.0, 'bldg_class_U7'), (33.0, 'DOB_complaint_2C'), (34.0, 'DOB_complaint_06'), (35.0, 'ECB_infr

Overall Feature Selection Model

In [17]:
ranks = {}
 
def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks ))
 
lr = LinearRegression(normalize=True)
lr.fit(X, Y)
ranks["Linear reg"] = rank_to_dict(np.abs(lr.coef_), names)
 
ridge = Ridge()
ridge.fit(X, Y)
ranks["Ridge"] = rank_to_dict(np.abs(ridge.coef_), names)

lasso = Lasso()
lasso.fit(X, Y)
ranks["Lasso"] = rank_to_dict(np.abs(lasso.coef_), names)
 
rlasso = RandomizedLasso()
rlasso.fit(X, Y)
ranks["Stability"] = rank_to_dict(np.abs(rlasso.scores_), names)

#stop the search when 5 features are left (they will get equal scores)
rfe = RFECV(lr, step = 1)
rfe.fit(X,Y)
ranks["RFE"] = rank_to_dict(map(float, rfe.ranking_), names, order=-1)
 
rf = RandomForestRegressor()
rf.fit(X,Y)
ranks["RF"] = rank_to_dict(rf.feature_importances_, names)
 
f, pval  = f_regression(X, Y, center=True)
ranks["Corr."] = rank_to_dict(f, names)
 
mine = MINE()
mic_scores = []
for i in range(X.shape[1]):
    mine.compute_score(X[:,i], Y)
    m = mine.mic()
    mic_scores.append(m)

ranks["MIC"] = rank_to_dict(mic_scores, names) 
 
r = {}
for name in names:
    r[name] = round(np.mean([ranks[method][name] for method in ranks.keys()]), 2)
    
methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")


In [18]:
ranksoutput = pd.DataFrame.from_dict(ranks)

In [21]:
ranksoutput.to_csv("Output of ranks.csv")